In [ ]:
!pip install langchain sentence-transformers transformers -q

In [ ]:
!CT_CUBLAS=1 pip install ctransformers --no-binary ctransformers -q

**Zero-Shot prompting**

In [ ]:
from langchain import PromptTemplate

In [ ]:
template='''clasify the text into neutral ,negative or positive
Text:{query}.
sentiment: '''

In [ ]:
prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

In [ ]:
prompt_template.format(
        query="I think the vacation is okay"
    )

In [ ]:
from langchain.llms import CTransformers
llm= CTransformers(model='TheBloke/Llama-2-13B-Chat-GGML',temperature=0, top_p=1,
    frequency_penalty=0,
    presence_penalty=0)

In [ ]:
from langchain import LLMChain

chain_1= LLMChain(llm=llm, prompt=prompt_template)
chain_1("i think Falcon 180B is big model,ima happy about it")

Note that in the prompt above we didn't provide the model with any examples -- that's the zero-shot capabilities at work. When zero-shot doesn't work, it's recommended to provide demonstrations or examples in the prompt and do few shot prompting technique

**Few shot prompting**
While large-language models already demonstrate remarkable zero-shot capabilities, they still fall short on more complex tasks when using the zero-shot setting. To improve on this, few-shot prompting is used as a technique to enable in-context learning where we provide demonstrations in the prompt to steer the model to better performance. The demonstrations serve as conditioning for subsequent examples where we would like the model to generate a response.

In [ ]:
from langchain.llms import CTransformers

llm = CTransformers(model='TheBloke/Llama-2-13B-Chat-GGML',temperature=0.6)

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain import PromptTemplate
from langchain import LLMChain
examples = [
  {
    "text": "this is awesome?",
    "sentiment": "positive"
  },
  {
    "text": "wow that movie was rad",
    "sentiment": "positive"
  },
     {
    "text": "what a horrible show",
    "sentiment": "negative"
  }
  ]

In [ ]:
# Next, we specify the template to format the examples we have provided.
# We use the `PromptTemplate` class for this.
example_formatter_template = """
Text: {text}\n{sentiment}"""


example_prompt = PromptTemplate(
    input_variables=["text", "sentiment"],
    template=example_formatter_template,
)
print(example_prompt.format(**examples[0]))

In [ ]:
# Finally, we create the `FewShotPromptTemplate` object.
few_shot_prompt = FewShotPromptTemplate(
    # These are the examples we want to insert into the prompt.
    examples=examples,
    # This is how we want to format the examples when we insert them into the prompt.
    example_prompt=example_prompt,
    # The prefix is some text that goes before the examples in the prompt.
    # Usually, this consists of intructions.
    prefix="Give the answer of every input\n",
    # The suffix is some text that goes after the examples in the prompt.
    # Usually, this is where the user input will go
    suffix="text: {input}\nsentiment: ",
    # The input variables are the variables that the overall prompt expects.
    input_variables=["input"],
    # The example_separator is the string we will use to join the prefix, examples, and suffix together with.
    example_separator="\n",
)     

In [ ]:
print(few_shot_prompt.format(input='what a peacefull show'))

In [ ]:
chain=LLMChain(llm=llm,prompt=few_shot_prompt)
chain({'input':"did you like movie"})

**Chain-Of-Thought(CoT) Prompting**
CoT provides a "chain of thought" process that showcases how the correct answer to a question should be reached. Provide the chain of thought either with examples (few-shot), or simply by adding "Let's think step by step" chain-of-thought (CoT) prompting enables complex reasoning capabilities through intermediate reasoning steps. You can combine it with few-shot prompting to get better results on more complex tasks that require reasoning before responding

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain import PromptTemplate
from langchain import LLMChain

In [ ]:
examples = [{
"Q":"The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.",
"A":"Adding all the odd numbers (17, 9, 13) gives 39. The answer is False."}
]

In [ ]:
# Next, we specify the template to format the examples we have provided.
# We use the `PromptTemplate` class for this.
example_formatter_template = """
Question: {Q}\n{A}"""


example_prompt = PromptTemplate(
    input_variables=["Q", "A"],
    template=example_formatter_template,
)
print(example_prompt.format(**examples[0]))

In [ ]:
# Finally, we create the `FewShotPromptTemplate` object.
few_shot_prompt = FewShotPromptTemplate(
    # These are the examples we want to insert into the prompt.
    examples=examples,
    # This is how we want to format the examples when we insert them into the prompt.
    example_prompt=example_prompt,
    # The prefix is some text that goes before the examples in the prompt.
    # Usually, this consists of intructions.
    prefix="Give the answer of every input\n",
    # The suffix is some text that goes after the examples in the prompt.
    # Usually, this is where the user input will go
    suffix="question: {input}\nanswer: ",
    # The input variables are the variables that the overall prompt expects.
    input_variables=["input"],
    # The example_separator is the string we will use to join the prefix, examples, and suffix together with.
    example_separator="\n",
)     

In [ ]:
print(few_shot_prompt.format(input='The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.'))

In [ ]:
chain=LLMChain(llm=llm,prompt=few_shot_prompt)
chain({'The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.'})

this is answer for CoT:Adding all the odd numbers (15, 5, 13, 7, 1) gives 41. The answer is False.